In [1]:
import pandas as pd

In [2]:
lfpr = pd.read_csv('LFPR_final.csv')
nfhs5 = pd.read_csv('merged_nfhs5_district_level.csv')
dom = pd.read_csv('nfhs5_female_dv_district_level_final.csv')

In [5]:
result = lfpr[['District', 'lfpr_percent']].merge(nfhs5[['district', 'state', 
                                                         'digital_gender_gap', 
                                                         'f_mean_age',
                                                         'f_mean_education',
                                                         'f_mean_yrs_edu',
                                                         'f_pct_literate',
                                                         'f_pct_sc',
                                                         'f_pct_st',
                                                         'f_pct_general',
                                                         'f_pct_obc',
                                                         'f_mean_wealth_scr',
                                                         'f_mean_wealth_idx',
                                                         'f_pct_urban',
                                                            'f_pct_rural',
                                                            'f_mean_hh_size',
                                                            'f_pct_married',
                                                            'f_pct_divorced',
                                                            'f_pct_has_mobile',
                                                            'f_pct_ever_breastfed_c1',
                                                            'f_n_respondents',




                                                         ]], on='district', how='left').to_csv('final.csv', index=False)

KeyError: 'district'

In [6]:
lfpr[['District', 'lfpr_percent']].rename(columns={'District': 'district'}).merge(nfhs5[['district', 'state', 'digital_gender_gap', 'f_mean_age', 'f_mean_education', 'f_mean_yrs_edu', 'f_pct_literate', 'f_pct_sc', 'f_pct_st', 'f_pct_general', 'f_pct_obc', 'f_mean_wealth_scr', 'f_mean_wealth_idx', 'f_pct_urban', 'f_pct_rural', 'f_mean_hh_size', 'f_pct_married', 'f_pct_divorced', 'f_pct_has_mobile', 'f_pct_ever_breastfed_c1', 'f_n_respondents']], on='district', how='outer').to_csv('final.csv', index=False)

In [9]:
import pandas as pd

# 1. Load the file generated in the previous step
final_df = pd.read_csv('final.csv')

# 2. Define the columns for male data based on the provided list
# Selected to match the themes in the female dataset (Labor, Digital, Demographics, Wealth, Caste)
male_columns = [
    'district',
    'm_mean_age',
    'm_mean_yrs_edu',
    'm_literacy_rate',          # Renamed from m_pct_literate
    'm_urban_share',            # Renamed from m_pct_urban
    'm_pct_sc',
    'm_pct_st',
    'm_pct_obc',
    'm_pct_general',
    'm_mean_wealth_idx',
    'm_mean_hh_size',
    'm_pct_married',
    'm_pct_owns_mobile',        # Matches digital/tech theme
    'm_pct_uses_internet',      # crucial for digital gender gap comparison
    'm_pct_currently_working',  # crucial for LFPR comparison
    'm_n_respondents'
]

# 3. Merge the new male data with the existing final dataframe
# Using 'outer' join to preserve all districts from both sides
result = final_df.merge(nfhs5[male_columns], on='district', how='outer')

# 4. Save the fully combined dataset
result.to_csv('final_combined_all.csv', index=False)

print("Merge complete. Saved to 'final_combined_all.csv'")

Merge complete. Saved to 'final_combined_all.csv'


In [10]:
import pandas as pd
import numpy as np

# 1. Load the dataset created in the previous step
df = pd.read_csv('final_combined_all.csv')

# 2. Handle missing respondent counts (fill with 0 to allow calculation, though usually NFHS has data)
df['f_n_respondents'] = df['f_n_respondents'].fillna(0)
df['m_n_respondents'] = df['m_n_respondents'].fillna(0)

# Calculate total respondents for weighting
df['total_respondents'] = df['f_n_respondents'] + df['m_n_respondents']

# 3. Define the pairs to combine (New Name : (Female Col, Male Col))
# Note: Using weighted average formula: (F_val * F_count + M_val * M_count) / Total_count
columns_to_combine = {
    'mean_age':        ('f_mean_age', 'm_mean_age'),
    'urban_share':     ('f_pct_urban', 'm_urban_share'), # Note: specific names from your list
    'pct_sc':          ('f_pct_sc', 'm_pct_sc'),
    'pct_st':          ('f_pct_st', 'm_pct_st'),
    'pct_obc':         ('f_pct_obc', 'm_pct_obc'),
    'pct_general':     ('f_pct_general', 'm_pct_general'),
    'mean_hh_size':    ('f_mean_hh_size', 'm_mean_hh_size'),
    'mean_wealth_idx': ('f_mean_wealth_idx', 'm_mean_wealth_idx')
}

# 4. Loop through and calculate the combined weighted columns
for new_col, (f_col, m_col) in columns_to_combine.items():
    # Ensure columns exist before processing
    if f_col in df.columns and m_col in df.columns:
        # Calculate weighted sum
        weighted_sum = (df[f_col] * df['f_n_respondents']) + (df[m_col] * df['m_n_respondents'])
        
        # Calculate weighted average (handle division by zero if total_respondents is 0)
        df[new_col] = np.where(
            df['total_respondents'] > 0, 
            weighted_sum / df['total_respondents'], 
            np.nan
        )
    else:
        print(f"Warning: Could not find columns {f_col} or {m_col}")

# 5. Save the result to a new file
# The prompt asked to "leave the rest of the columns the same", so we keep everything plus the new columns.
output_filename = 'final_aggregated.csv'
df.to_csv(output_filename, index=False)

print(f"Aggregated columns created. Saved to {output_filename}")
print("New columns added:", list(columns_to_combine.keys()))

Aggregated columns created. Saved to final_aggregated.csv
New columns added: ['mean_age', 'urban_share', 'pct_sc', 'pct_st', 'pct_obc', 'pct_general', 'mean_hh_size', 'mean_wealth_idx']


In [11]:
import pandas as pd

# 1. Load the datasets
df_final = pd.read_csv('final_aggregated.csv')
df_sorted = pd.read_csv('sorted_data_2.csv')

# 2. Select the specific columns to add from sorted_data_2
# We include 'district' so we have a common key to merge on
cols_to_add = ['district', 'dd_mobile_own_5', 'dd_mobile_own_4', 'dd_gap', 'treatment_status']
df_sorted_subset = df_sorted[cols_to_add]

# 3. Merge the dataframes based on 'district'
# how='left' keeps all rows from final_aggregated.csv and adds matching data from sorted_data_2
merged_df = pd.merge(df_final, df_sorted_subset, on='district', how='left')

# 4. Save the result to a new CSV file
merged_df.to_csv('final_aggregated_updated.csv', index=False)

# Display the first few rows to verify
print(merged_df.head())

     district  lfpr_percent           state  digital_gender_gap  f_mean_age  \
0    adilabad     32.653061       telangana           50.138683   30.608635   
1  agar malwa           NaN  madhya pradesh           70.384850   30.799053   
2        agra      5.569007   uttar pradesh           47.109463   29.445339   
3   ahmadabad     14.343029         gujarat           26.449638   31.456207   
4  ahmadnagar           NaN     maharashtra           51.732426   30.833387   

   f_mean_education  f_mean_yrs_edu  f_pct_literate   f_pct_sc   f_pct_st  \
0          1.354055        6.618928       65.587166  16.131357  29.970905   
1          1.153084        5.260029       64.495285  19.424776   2.978825   
2          1.453236        7.003464       71.480148  32.112534   0.727786   
3          1.772380        8.402651       84.075867  21.410913   7.357196   
4          1.826169        8.640099       88.874695  11.696818  10.957435   

   ...     pct_sc     pct_st    pct_obc  pct_general  mean_hh_

In [ ]:
import pandas as pd

# 1. Load the datasets
df_final = pd.read_csv('final_aggregated_updated.csv')
df_sorted = pd.read_csv('nfhs5_female_dv_district_level_final.csv')

# 2. Select the specific columns to add from sorted_data_2
# We include 'district' so we have a common key to merge on
cols_to_add = ['district', 'f_mean_bf_dur_c1']
df_sorted_subset = df_sorted[cols_to_add]

# 3. Merge the dataframes based on 'district'
# how='left' keeps all rows from final_aggregated.csv and adds matching data from sorted_data_2
merged_df = pd.merge(df_final, df_sorted_subset, on='district', how='left')

# 4. Save the result to a new CSV file
merged_df.to_csv('newer.csv', index=False)

# Display the first few rows to verify
print(merged_df.head())

     district  lfpr_percent           state  digital_gender_gap  f_mean_age  \
0    adilabad     32.653061       telangana           50.138683   30.608635   
1  agar malwa           NaN  madhya pradesh           70.384850   30.799053   
2        agra      5.569007   uttar pradesh           47.109463   29.445339   
3   ahmadabad     14.343029         gujarat           26.449638   31.456207   
4  ahmadnagar           NaN     maharashtra           51.732426   30.833387   

   f_mean_education  f_mean_yrs_edu  f_pct_literate   f_pct_sc   f_pct_st  \
0          1.354055        6.618928       65.587166  16.131357  29.970905   
1          1.153084        5.260029       64.495285  19.424776   2.978825   
2          1.453236        7.003464       71.480148  32.112534   0.727786   
3          1.772380        8.402651       84.075867  21.410913   7.357196   
4          1.826169        8.640099       88.874695  11.696818  10.957435   

   ...    pct_obc  pct_general  mean_hh_size  mean_wealth_idx 

In [16]:
import pandas as pd

# 1. Load your datasets
df1 = pd.read_csv('final_aggregated_updated.csv')
df2 = pd.read_csv('nfhs5_female_dv_district_level_final.csv')

# 2. Merge the datasets
# 'how' can be: 'inner', 'left', 'right', or 'outer'
merged_df = pd.merge(df1, df2, on='district', how='inner')

# 3. View and Save the result
print(merged_df.head())
merged_df.to_csv('ultimate.csv', index=False)

     district  lfpr_percent           state  digital_gender_gap  f_mean_age  \
0    adilabad     32.653061       telangana           50.138683   30.608635   
1  agar malwa           NaN  madhya pradesh           70.384850   30.799053   
2        agra      5.569007   uttar pradesh           47.109463   29.445339   
3   ahmadabad     14.343029         gujarat           26.449638   31.456207   
4  ahmadnagar           NaN     maharashtra           51.732426   30.833387   

   f_mean_education  f_mean_yrs_edu  f_pct_literate   f_pct_sc   f_pct_st  \
0          1.354055        6.618928       65.587166  16.131357  29.970905   
1          1.153084        5.260029       64.495285  19.424776   2.978825   
2          1.453236        7.003464       71.480148  32.112534   0.727786   
3          1.772380        8.402651       84.075867  21.410913   7.357196   
4          1.826169        8.640099       88.874695  11.696818  10.957435   

   ...  m_freq_hit_last12m_non_hp  m_hp_hurt_during_pregnancy 